In [ ]:
# 随机选择一辆车
import random
import torch
mask_veh = torch.randint(2, (1, 61)).bool()

index = torch.arange(0, 61)

available_index = torch.masked_select(index, ~mask_veh)

random_index = random.choice(available_index)[None]

mask_veh, index, available_index, random_index

(tensor([[False, False, False, False,  True,  True,  True,  True, False,  True,
          False, False,  True,  True, False, False,  True, False, False,  True,
           True,  True, False,  True, False,  True, False, False,  True,  True,
          False, False, False,  True,  True, False,  True, False,  True,  True,
          False, False,  True, False,  True,  True, False, False, False,  True,
          False,  True,  True, False, False,  True, False,  True,  True,  True,
           True]]),
 tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
         36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
         54, 55, 56, 57, 58, 59, 60]),
 tensor([ 0,  1,  2,  3,  8, 10, 11, 14, 15, 17, 18, 22, 24, 26, 27, 30, 31, 32,
         35, 37, 40, 41, 43, 46, 47, 48, 50, 53, 54, 56]),
 tensor([3]))

In [95]:
# simulate_demo_1
# [batch_size, graph_node, embedding_size], [2, 3, 6]
embeddings = torch.tensor([[[ 0.7719,  1.4574,  0.4904,  -1.4429,  1.3310,  1.2217],
         [ 0.7716,  1.4572,  0.4895,   -1.4425,  1.3301,  1.2213],
         [ 0.7719,  1.4574,  0.4904,   -1.4429,  1.3310,  1.2217]],
         [[ 0.8358, -0.5840, -0.8632,    0.6978, -1.2706, -1.4171],
         [ 0.8043, -0.8615, -0.6384,    0.7062, -1.6867, -2.2006],
         [ 0.5360, -0.2593, -0.9840,    0.7619, -0.4540, -0.2393]]],
       device='cuda:0')
current_node = torch.tensor([[ 0,  1,  2], [0,  1,  2]], device='cuda:0')   # 所有车辆上一步所在结点
batch_size, num_veh = current_node.size()   # 2, 3
capacity = torch.tensor([[1., 1., 1.],[4., 4., 4.]], device='cuda:0')   # 所有车辆原始容量
used_capacity = torch.tensor([[0., 1., 0.], [0., 4., 3.]], device='cuda:0') # 所有车辆已使用容量
veh_passed_time = torch.tensor([[100., 0., 50.], [20., 40., 30.]], device='cuda:0') # 所有车辆已行驶时间
veh = torch.tensor([0, 1], device='cuda:0') # 当前步选定车辆

time = torch.zeros((batch_size, 1, 1), device=veh_passed_time.device)
for i in range(0, veh.size(-1)):
    time[i, 0, 0] = veh_passed_time[i, veh[i]]
print('context_time:', time) # tensor([[[100.]], [[ 40.]]], device='cuda:0') [2, 1, 1]
o_t = (capacity[torch.arange(batch_size), veh] - used_capacity[torch.arange(batch_size), veh]).unsqueeze(0).transpose(0, 1).unsqueeze(-1)
print('context_o:', o_t) # tensor([[[1.]], [[0.]]], device='cuda:0') [2, 1, 1]


index = current_node[torch.arange(batch_size), veh].contiguous().view(batch_size, 1, 1).expand(batch_size, 1, embeddings.size(-1)) 
print("index:", index) # tensor([[[0, 0, 0, 0, 0, 0]], [[1, 1, 1, 1, 1, 1]]], device='cuda:0')

torch.cat(
    (
        torch.gather(embeddings, 1, index),
        o_t,
        time
    )
    ,-1)

context_time: tensor([[[100.]],

        [[ 40.]]], device='cuda:0')
context_o: tensor([[[1.]],

        [[0.]]], device='cuda:0')
index: tensor([[[0, 0, 0, 0, 0, 0]],

        [[1, 1, 1, 1, 1, 1]]], device='cuda:0')


tensor([[[  0.7719,   1.4574,   0.4904,  -1.4429,   1.3310,   1.2217,   1.0000,
          100.0000]],

        [[  0.8043,  -0.8615,  -0.6384,   0.7062,  -1.6867,  -2.2006,   0.0000,
           40.0000]]], device='cuda:0')

In [106]:
import os
import torch
from haversine import haversine

def gps_distance(pt1, pt2):
    r = haversine((pt1[1], pt1[0]), (pt2[1], pt2[0]), 'm')     # (latitude, longitude)   默认单位为km
    return r

# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# simulate_demo_2
# [batch_size, graph_node, embedding_size], [2, 23, 6] veh_node:3 graph_node:10 new_graph_node:10
embeddings = torch.tensor([[[ 0.7711,  1.4574,  0.4904,  -1.4429,  1.3310,  1.2217],
         [ 0.7712,  1.4572,  0.4895,   -1.4425,  1.3301,  1.2213],
         [ 0.7713,  1.4574,  0.4904,   -1.4429,  1.3310,  1.2217],
         [ 0.7714,  1.4574,  0.4904,  -1.4429,  1.3310,  1.2217],
         [ 0.7715,  1.4572,  0.4895,   -1.4425,  1.3301,  1.2213],
         [ 0.7716,  1.4574,  0.4904,   -1.4429,  1.3310,  1.2217],
         [ 0.7717,  1.4574,  0.4904,  -1.4429,  1.3310,  1.2217],
         [ 0.7718,  1.4572,  0.4895,   -1.4425,  1.3301,  1.2213],
         [ 0.7719,  1.4574,  0.4904,   -1.4429,  1.3310,  1.2217],
         [ 0.7710,  1.4574,  0.4904,  -1.4429,  1.3310,  1.2217],
         [ 0.7726,  1.4572,  0.4895,   -1.4425,  1.3301,  1.2213],
         [ 0.7319,  1.4574,  0.4904,  -1.4429,  1.3310,  1.2217],
         [ 0.7746,  1.4572,  0.4895,   -1.4425,  1.3301,  1.2213],
         [ 0.9719,  1.4574,  0.4904,  -1.4429,  1.3310,  1.2217],
         [ 0.5716,  1.4572,  0.4895,   -1.4425,  1.3301,  1.2213],
         [ 0.4719,  1.4574,  0.4904,   -1.4429,  1.3310,  1.2217],
         [ 0.5719,  1.4574,  0.4904,  -1.4429,  1.3310,  1.2217],
         [ 0.7716,  1.4572,  0.4895,   -1.4425,  1.3301,  1.2213],
         [ 0.3719,  1.4574,  0.4904,   -1.4429,  1.3310,  1.2217],
         [ 0.2719,  1.4574,  0.4904,  -1.4429,  1.3310,  1.2217],
         [ 0.7816,  1.4572,  0.4895,   -1.4425,  1.3301,  1.2213],
         [ 0.7719,  1.4574,  0.4904,  -1.4429,  1.3310,  1.2217],
         [ 0.7726,  1.4572,  0.4895,   -1.4425,  1.3301,  1.2213]],
         [[ 0.7711,  1.4574,  0.4904,  -1.4429,  1.3310,  1.2217],
         [ 0.7712,  1.4572,  0.4895,   -1.4425,  1.3301,  1.2213],
         [ 0.7713,  1.4574,  0.4904,   -1.4429,  1.3310,  1.2217],
         [ 0.7714,  1.4574,  0.4904,  -1.4429,  1.3310,  1.2217],
         [ 0.7715,  1.4572,  0.4895,   -1.4425,  1.3301,  1.2213],
         [ 0.7716,  1.4574,  0.4904,   -1.4429,  1.3310,  1.2217],
         [ 0.7717,  1.4574,  0.4904,  -1.4429,  1.3310,  1.2217],
         [ 0.7718,  1.4572,  0.4895,   -1.4425,  1.3301,  1.2213],
         [ 0.7719,  1.4574,  0.4904,   -1.4429,  1.3310,  1.2217],
         [ 0.7710,  1.4574,  0.4904,  -1.4429,  1.3310,  1.2217],
         [ 0.7726,  1.4572,  0.4895,   -1.4425,  1.3301,  1.2213],
         [ 0.7319,  1.4574,  0.4904,  -1.4429,  1.3310,  1.2217],
         [ 0.7746,  1.4572,  0.4895,   -1.4425,  1.3301,  1.2213],
         [ 0.9719,  1.4574,  0.4904,  -1.4429,  1.3310,  1.2217],
         [ 0.5716,  1.4572,  0.4895,   -1.4425,  1.3301,  1.2213],
         [ 0.4719,  1.4574,  0.4904,   -1.4429,  1.3310,  1.2217],
         [ 0.5719,  1.4574,  0.4904,  -1.4429,  1.3310,  1.2217],
         [ 0.7716,  1.4572,  0.4895,   -1.4425,  1.3301,  1.2213],
         [ 0.3719,  1.4574,  0.4904,   -1.4429,  1.3310,  1.2217],
         [ 0.2719,  1.4574,  0.4904,  -1.4429,  1.3310,  1.2217],
         [ 0.7816,  1.4572,  0.4895,   -1.4425,  1.3301,  1.2213],
         [ 0.7719,  1.4574,  0.4904,  -1.4429,  1.3310,  1.2217],
         [ 0.7726,  1.4572,  0.4895,   -1.4425,  1.3301,  1.2213]]],device='cuda:0')
print("embedding's size:", embeddings.size())
coords = torch.tensor([[[-122.4100772,   37.7880844],
         [-122.3950575,   37.777057],
         [-122.4071627,   37.779736],
         
         [-122.40956,   37.7879],
         [-122.40863,   37.78982],
         [-122.41362,   37.78674],
         [-122.41982,   37.78674],
         [-122.39538,   37.77667],
         
         [-122.39273,   37.77874],
         [-122.39538,   37.77667],
         [-122.39566,   37.77646],
         [-122.39515,   37.78087],
         [-122.40899,   37.78119],
         
         [-122.38877,   37.77949],
         [-122.41636,   37.78577],
         [-122.39318,   37.78221],
         [-122.40563,   37.75687],
         [-122.39738,   37.79452],
         
         [-122.40396,   37.77370],
         [-122.42042,   37.77973],
         [-122.39703,   37.78874],
         [-122.39477,   37.78900],
         [-122.41298,   37.76872]],
                       
         [[-122.4100772,   37.7880844],
         [-122.3950575,   37.777057],
         [-122.4071627,   37.779736],
         
         [-122.40956,   37.7879],
         [-122.40863,   37.78982],
         [-122.41362,   37.78674],
         [-122.41982,   37.78674],
         [-122.39538,   37.77667],
         
         [-122.39273,   37.77874],
         [-122.39538,   37.77667],
         [-122.39566,   37.77646],
         [-122.39515,   37.78087],
         [-122.40899,   37.78119],
         
         [-122.38877,   37.77949],
         [-122.41636,   37.78577],
         [-122.39318,   37.78221],
         [-122.40563,   37.75687],
         [-122.39738,   37.79452],
         
         [-122.40396,   37.77370],
         [-122.42042,   37.77973],
         [-122.39703,   37.78874],
         [-122.39477,   37.78900],
         [-122.41298,   37.76872]]],device='cuda:0')
print("coords's size:", coords.size())
# current_route = torch.tensor(
#     [[[0, 4, 3, 13, 14],  
#       [1, 7, 17],  
#       [2, 6, 5, 15, 16]]], device='cuda:0')   # 所有车辆上一步的路线

current_route = [[[0, 4, 3, 13, 14],  
      [1, 7, 8, 17, 9],  
      [2, 6, 5, 15, 16]],
      [[0, 4, 3, 13, 14],  
      [1, 7, 17],  
      [2, 6, 9, 5, 15, 16]]]

current_route_loc =  [[[] for j in range(num_veh)] for i in range(batch_size)]
for i in range(0, batch_size):
      for j in range(0, num_veh):
            for k in range(0, len(current_route[i][j])):
                  current_route_loc[i][j].append(coords[i, current_route[i][j][k]])
# print("current_route_loc:",current_route_loc)

current_route_time_cost = [[[] for j in range(num_veh)] for i in range(batch_size)]
# init_route_time
SPEED = 1000 / 3600
for i in range(0, batch_size):
      for j in range(0, len(current_route_loc[i])):
            for k in range(0, len(current_route_loc[i][j]) - 1):
                  time_cost = gps_distance(current_route_loc[i][j][k], current_route_loc[i][j][k+1]) / SPEED
                  current_route_time_cost[i][j].append(time_cost)
# print("current_route_time_cost:",current_route_time_cost)

batch_size, num_veh = len(current_route), len(current_route[0])
print("batch_size:", batch_size, "num_veh:", num_veh)
capacity = torch.tensor([[4., 4., 4.], [4., 4., 4.]], device='cuda:0')   # 所有车辆原始容量
used_capacity = torch.tensor([[2., 3., 2.], [2., 1., 3.]], device='cuda:0') # 所有车辆已使用容量
# veh_passed_time = torch.tensor([[100., 0., 50.], [20., 40., 30.]], device='cuda:0') # 所有车辆已行驶时间?
veh = torch.tensor([1, 2], device='cuda:0') # 当前步选定车辆 选定1号车

# time = torch.zeros((batch_size, 1, 1), device=veh_passed_time.device) # time应该怎么拼接？
# for i in range(0, veh.size(-1)):
#     time[i, 0, 0] = veh_passed_time[i, veh[i]]
# print('context_time:', time) # tensor([[[100.]], [[ 40.]]], device='cuda:0') [2, 1, 1]
# o_t = (capacity[torch.arange(batch_size), veh] - used_capacity[torch.arange(batch_size), veh]).unsqueeze(0).transpose(0, 1).unsqueeze(-1)
# print('context_o:', o_t) # tensor([[[1.]], [[0.]]], device='cuda:0') [2, 1, 1]

# compute query
route_index = []
for i in range(0, veh.size(-1)):
      route = torch.tensor(current_route[i][veh[i]], device='cuda:0')
      route_index.append(route.contiguous().view(route.size(-1), 1).expand(-1, embeddings.size(-1)))
print("route_index:", route_index)

route_embeddings = []
for index, value in enumerate(route_index):
      route_embeddings.append(torch.gather(embeddings[index, :, :], 0, value))
print("route_embeddings:", route_embeddings)

embedding's size: torch.Size([2, 23, 6])
coords's size: torch.Size([2, 23, 2])
batch_size: 2 num_veh: 3
route_index: [tensor([[ 1,  1,  1,  1,  1,  1],
        [ 7,  7,  7,  7,  7,  7],
        [ 8,  8,  8,  8,  8,  8],
        [17, 17, 17, 17, 17, 17],
        [ 9,  9,  9,  9,  9,  9]], device='cuda:0'), tensor([[ 2,  2,  2,  2,  2,  2],
        [ 6,  6,  6,  6,  6,  6],
        [ 9,  9,  9,  9,  9,  9],
        [ 5,  5,  5,  5,  5,  5],
        [15, 15, 15, 15, 15, 15],
        [16, 16, 16, 16, 16, 16]], device='cuda:0')]
route_embeddings: [tensor([[ 0.7712,  1.4572,  0.4895, -1.4425,  1.3301,  1.2213],
        [ 0.7718,  1.4572,  0.4895, -1.4425,  1.3301,  1.2213],
        [ 0.7719,  1.4574,  0.4904, -1.4429,  1.3310,  1.2217],
        [ 0.7716,  1.4572,  0.4895, -1.4425,  1.3301,  1.2213],
        [ 0.7710,  1.4574,  0.4904, -1.4429,  1.3310,  1.2217]],
       device='cuda:0'), tensor([[ 0.7713,  1.4574,  0.4904, -1.4429,  1.3310,  1.2217],
        [ 0.7717,  1.4574,  0.4904, -1.44

In [107]:
# compute glimpse_key and glimpse_value

# how to mask node?
visited = torch.tensor([[[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]],
                        [[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]]],
       device='cuda:0', dtype=torch.uint8)
demand = torch.tensor([[ 0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  
                         0.,  0.,  0.,  0.,  0., -1., -1., -1., -1., -1.],
                       [ 0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  
                         0.,  0.,  0.,  0.,  0., -1., -1., -1., -1., -1.]],
       device='cuda:0')
time = torch.tensor([[ 0.,  0.,  0.,  0.,  0.,  10000.,  10000.,  10000.,  15000.,  20000.,  
                         0.,  0.,  0.,  0.,  0., 20000., 20000., 20000., 30000., 40000.],
                       [ 0.,  0.,  0.,  0.,  0., 10000.,  10000.,  10000.,  15000.,  20000.,  
                         0.,  0.,  0.,  0.,  0., 20000., 20000., 20000., 30000., 40000.]],
       device='cuda:0')
to_delivery = torch.tensor([[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                             [1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0],
                             [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
                            [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                             [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                             [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]]], 
        device='cuda:0')
ids=torch.arange(batch_size, dtype=torch.int64, device=demand.device)[:, None]
print("visited's size:", visited.size(), "demand's size:", demand.size(), "to_delivery's size:", to_delivery.size())

visited_loc = visited[:, :, :]
batch_size = visited_loc.size(0)
pre_route = current_route

invalid_demand = []
exceeds_cap = []
delivery_for_veh = []
mask = []
for i in range(0, batch_size):
    graph_size = demand[i, :].size(-1)
    num_veh = capacity[i, :].size(-1)
    # print("graph_size:", graph_size, "num_veh:", num_veh)

    # capacity
    plan_route = current_route[i][veh[i]] # [1, 7, 17]
    plan_route_loc = current_route_loc[i][veh[i]]
    plan_route_time = current_route_time_cost[i][veh[i]]
    plan_route_acc_time = plan_route_time[:]
    for k in range(1, len(plan_route_acc_time)):
        plan_route_acc_time[k] += plan_route_acc_time[k-1]

    if i == 0:
        print("plan_route:", plan_route)
        print("plan_route_loc:", plan_route_loc)
        print("plan_route_time:", plan_route_time)
        print("plan_route_acc_time:", plan_route_acc_time)
    plan_route_length = len(plan_route)
    
    cur_capacity = (capacity[i, veh[i]] - used_capacity[i, veh[i]]).repeat(plan_route_length, 1).squeeze(1)
    for k in range(1, plan_route_length):
        if (plan_route[k] >= (num_veh + graph_size // 2)) and (plan_route[k] < (num_veh + graph_size)):
            cur_capacity[k] = cur_capacity[k-1] + 1
    # print("current_capacity:", cur_capacity.size(), cur_capacity)

    # demand == 0
    invalid_demand.append((demand[ids[i], :] == 0)[..., None].repeat(1, 1, plan_route_length)) # [1, graph_size, plan_route_length]

    # demand > remain capacity
    route_demand = demand[ids[i], :].unsqueeze(-1).expand(-1, -1, plan_route_length)
    exceeds_cap.append(route_demand > cur_capacity[None, None, :].expand_as(route_demand))

    # is_to_delivery: delivery node can only be inserted after the corresponding pickup node 
    cur_delivery = to_delivery[i, veh[i], :][None, :, None].repeat(1, 1, plan_route_length)
    pick_index = (1 - to_delivery[i, veh[i], :])[0:graph_size//2].nonzero().squeeze(-1).tolist()
    for pick_node in pick_index:
        k = plan_route.index(pick_node + num_veh)
        cur_delivery[:, pick_node + graph_size//2, 0:k] = 0
    delivery_for_veh.append(1 - cur_delivery)

    # is_visited
    is_visited = visited_loc[i].unsqueeze(-1).expand(-1, -1, plan_route_length)
    
    mask_pos = is_visited | invalid_demand[i] | exceeds_cap[i] | delivery_for_veh[i]
    print(mask_pos)
    
    # time
    avail_pos = (1 - mask_pos).nonzero()
    for k in range(0, avail_pos.size(0)): # [batch_id, node_index, insert_position]
        insert_pos = avail_pos[k, 2]
        insert_node = avail_pos[k, 1] # coords中的坐标包含车辆的初始结点
        time_cost = gps_distance(plan_route_loc[insert_pos], coords[i, insert_node+num_veh, :]) / SPEED # 需要抛异常吗
        if insert_pos > 0:
            time_cost += plan_route_acc_time[insert_pos-1]
        if insert_node < graph_size//2 and time_cost > time[i, insert_node]:
            mask_pos[:, insert_node, insert_pos] = 1
    
    mask.append(mask_pos)


visited's size: torch.Size([2, 1, 20]) demand's size: torch.Size([2, 20]) to_delivery's size: torch.Size([2, 3, 20])
plan_route: [1, 7, 8, 17, 9]
plan_route_loc: [tensor([-122.3951,   37.7771], device='cuda:0'), tensor([-122.3954,   37.7767], device='cuda:0'), tensor([-122.3927,   37.7787], device='cuda:0'), tensor([-122.3974,   37.7945], device='cuda:0'), tensor([-122.3954,   37.7767], device='cuda:0')]
plan_route_time: [185.8476260346732, 1178.6249395332716, 6486.082766963793, 7174.434205410446]
plan_route_acc_time: [185.8476260346732, 1364.4725655679447, 7850.555332531738, 15024.989537942183]
tensor([[[1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1],
       

In [123]:
import copy
# route embedding after insert new node
final_avail_pos = []
for mask_pos in mask:
    final_avail_pos.append((1-mask_pos).nonzero())
print(final_avail_pos)

# compute key and value
insert_route_index = [[] for i in range(veh.size(-1))]
for i in range(0, veh.size(-1)):
      for k in range(0, final_avail_pos[i].size(0)):
            plan_route = current_route[i][veh[i]].copy() # 需要向嵌套列表中插入新的结点 计算所有可能的路线 不能修改原来的计划路线
            insert_index = final_avail_pos[i][k, 2].item() + 1
            insert_node = final_avail_pos[i][k, 1].item() + num_veh
            # print(insert_index, insert_node)
            plan_route.insert(insert_index, insert_node)
            insert_route = torch.tensor(plan_route, device='cuda:0')
            insert_route_index[i].append(insert_route.contiguous().view(insert_route.size(-1), 1).expand(-1, embeddings.size(-1)))
print("insert_route_index:", insert_route_index)

insert_route_embeddings = [[] for i in range(veh.size(-1))]
for i in range(0, veh.size(-1)):
      for value in insert_route_index[i]:
            insert_route_embeddings[i].append(torch.gather(embeddings[i, :, :], 0, value))
print("insert_route_embeddings:", insert_route_embeddings)

[tensor([[ 0,  7,  0],
        [ 0,  7,  1],
        [ 0,  7,  2],
        [ 0,  8,  0],
        [ 0,  8,  1],
        [ 0,  8,  2],
        [ 0,  8,  3],
        [ 0,  9,  0],
        [ 0,  9,  1],
        [ 0,  9,  2],
        [ 0,  9,  3],
        [ 0,  9,  4],
        [ 0, 15,  2],
        [ 0, 15,  3],
        [ 0, 15,  4],
        [ 0, 16,  4]], device='cuda:0'), tensor([[ 0,  5,  0],
        [ 0,  7,  0],
        [ 0,  8,  0],
        [ 0,  8,  1],
        [ 0,  9,  0],
        [ 0,  9,  1],
        [ 0,  9,  2],
        [ 0, 16,  2],
        [ 0, 16,  3],
        [ 0, 16,  4],
        [ 0, 16,  5]], device='cuda:0')]
insert_route_index: [[tensor([[ 1,  1,  1,  1,  1,  1],
        [10, 10, 10, 10, 10, 10],
        [ 7,  7,  7,  7,  7,  7],
        [ 8,  8,  8,  8,  8,  8],
        [17, 17, 17, 17, 17, 17],
        [ 9,  9,  9,  9,  9,  9]], device='cuda:0'), tensor([[ 1,  1,  1,  1,  1,  1],
        [ 7,  7,  7,  7,  7,  7],
        [10, 10, 10, 10, 10, 10],
        [ 8,  8,  8,

In [93]:
x1 = torch.randn(1,3)
x2 = torch.randn(1,3)
x1, x2, x1 + x2

x = torch.tensor([3., 3., 3., 3.], device='cuda:0')
for i in range(1, x.size(-1)):
    x[i] = x[i-1] + 1
x

tensor([3., 4., 5., 6.], device='cuda:0')

In [132]:
vh_request = {
    148: [{'id': 34987, 'o_x': -122.44013999999999, 'o_y': 37.79652, 'd_x': -122.43966, 'd_y': 37.78468, 'status': 'unassigned'}, 
          {'id': 422238, 'o_x': -122.43321999999999, 'o_y': 37.79743, 'd_x': -122.41548, 'd_y': 37.778940000000006, 'status': 'unassigned'}, 
          {'id': 254292, 'o_x': -122.43468, 'o_y': 37.7972, 'd_x': -122.41851000000001, 'd_y': 37.77202, 'status': 'unassigned'}], 
    335: [{'id': 34987, 'o_x': -122.44013999999999, 'o_y': 37.79652, 'd_x': -122.43966, 'd_y': 37.78468, 'status': 'unassigned'}, 
          {'id': 312484, 'o_x': -122.39515, 'o_y': 37.78087, 'd_x': -122.39477, 'd_y': 37.789, 'status': 'unassigned'}], 
    416: [{'id': 57379, 'o_x': -122.39538, 'o_y': 37.77667, 'd_x': -122.39738, 'd_y': 37.79452, 'status': 'unassigned'}, 
          {'id': 123043, 'o_x': -122.40700000000001, 'o_y': 37.78942, 'd_x': -122.3972, 'd_y': 37.79367, 'status': 'unassigned'}, 
          {'id': 164196, 'o_x': -122.39703, 'o_y': 37.785579999999996, 'd_x': -122.39438999999999, 'd_y': 37.794129999999996,  'status': 'unassigned'}]
    }
vehicles = [148, 335, 416]
graph = (
    {'id': 34987, 'o_x': -122.44013999999999, 'o_y': 37.79652, 'd_x': -122.43966, 'd_y': 37.78468, 'status': 'unassigned'},
    {'id': 422238, 'o_x': -122.43321999999999, 'o_y': 37.79743, 'd_x': -122.41548, 'd_y': 37.778940000000006, 'status': 'unassigned'},
    {'id': 254292, 'o_x': -122.43468, 'o_y': 37.7972, 'd_x': -122.41851000000001, 'd_y': 37.77202, 'status': 'unassigned'},
    {'id': 312484, 'o_x': -122.39515, 'o_y': 37.78087, 'd_x': -122.39477, 'd_y': 37.789, 'status': 'unassigned'},
    {'id': 57379, 'o_x': -122.39538, 'o_y': 37.77667, 'd_x': -122.39738, 'd_y': 37.79452, 'status': 'unassigned'},
    {'id': 123043, 'o_x': -122.40700000000001, 'o_y': 37.78942, 'd_x': -122.3972, 'd_y': 37.79367, 'status': 'unassigned'},
    {'id': 164196, 'o_x': -122.39703, 'o_y': 37.785579999999996, 'd_x': -122.39438999999999, 'd_y': 37.794129999999996,  'status': 'unassigned'}
)

to_visited = torch.zeros(len(vehicles), len(graph)*2)
print(to_visited)

for veh, requests in vh_request.items():
      veh_index = vehicles.index(veh)
      for req in requests:
            req_index = graph.index(req)
            to_visited[veh_index, req_index] = 1
print(to_visited)

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.]])


In [136]:
# select_node

probs = torch.tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 6.5123e-02,
         0.0000e+00, 0.0000e+00, 6.0046e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 1.3835e-04, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 2.0752e-06, 0.0000e+00, 2.0252e-04, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 2.4002e-04, 0.0000e+00,
         8.6334e-05, 0.0000e+00, 2.4989e-04, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 3.3174e-01, 3.3238e-05, 0.0000e+00, 2.7028e-04,
         0.0000e+00, 0.0000e+00, 1.3705e-04, 0.0000e+00, 1.4655e-04, 0.0000e+00,
         6.8412e-07, 0.0000e+00, 2.7100e-06, 2.3945e-06, 0.0000e+00, 6.5161e-05,
         5.1309e-05, 0.0000e+00, 0.0000e+00, 0.0000e+00, 3.8323e-06, 4.2844e-05,
         0.0000e+00, 1.0670e-04, 0.0000e+00, 9.4663e-05, 1.0154e-04, 0.0000e+00,
         0.0000e+00, 6.3411e-04, 6.5550e-05, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00]],
       device='cuda:0')
value,index = probs.max(1)
value, index, probs.multinomial(1).squeeze(1)

(tensor([0.6005], device='cuda:0'),
 tensor([8], device='cuda:0'),
 tensor([8], device='cuda:0'))

In [185]:
x = torch.tensor([1, 1, 1])
y = torch.tensor([1, 1, 0])

print(y.nonzero().squeeze(-1))
visited = torch.tensor([[[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]],
       device='cuda:0', dtype=torch.uint8)
z = torch.tensor([[2], [18], [33]])
visited[0, :, z]
mask = torch.tensor([True, False])
if mask.all():
    print('haha ')

tensor([0, 1])


In [214]:
import random
batch_size = 2
num_veh = 3
mask = torch.tensor([[True, False, False], [False, False, False]])
vehicles = torch.arange(0, num_veh, dtype=torch.int64)
available_vehicles = torch.arange(0, batch_size, dtype=torch.int64)
for i in range(batch_size):
    available_vehicles[i] = random.choice(vehicles[~mask[i]])
available_vehicles

tensor([2, 1])

In [217]:
veh = torch.tensor([93, 66])
veh.size()

torch.Size([2])

In [10]:
import torch
batch_size = 2
num_veh = 3
graph_size = 5
length = 4
veh = torch.tensor([0, 1])
to_visit = torch.tensor([[[1, 1, 1, 0, 0],[1, 0, 0, 1, 1],[0, 0, 1, 1, 1]],[[1, 1, 1, 0, 0],[1, 0, 0, 1, 1],[0, 0, 1, 1, 1]]])
for i in range(batch_size):
    print(to_visit[i, veh[i], :][None, :, None].expand(-1, -1, length).size())

torch.Size([1, 5, 4])
torch.Size([1, 5, 4])


In [31]:
final_avail_pos = [torch.tensor([[ 0,  2,  0],
        [ 0, 18,  0],
        [ 0, 20,  0],
        [ 0, 24,  0],
        [ 0, 36,  0],
        [ 0, 40,  0],
        [ 0, 44,  0],
        [ 0, 50,  0],
        [ 0, 57,  0],
        [ 0, 68,  0]], device='cuda:0')]
probs = [torch.tensor([[ 0.0250, -0.0621, -0.0477, -0.0304, -0.0158, -0.0028, -0.0179, -0.1133,
          0.0742, -0.0226]], device='cuda:0')]
selected = []
batch_size = len(final_avail_pos)
for i in range(batch_size):
    selected.append(final_avail_pos[i][probs[i].exp()[0, :].max(0)[1]])
    selected.append((final_avail_pos[i][probs[i].exp()[0, :].multinomial(1)]).squeeze(0))
selected

[tensor([ 0, 57,  0], device='cuda:0'), tensor([ 0, 57,  0], device='cuda:0')]

In [1]:
import torch
veh = torch.tensor([16, 20], device='cuda:0')
batch_size = 2
prev_a = torch.tensor([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
          14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
          28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
          42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
          56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
          70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
          84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
          98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
         112, 113, 114, 115, 116, 117, 118, 119, 120, 121],
         [  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
          14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
          28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
          42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
          56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
          70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
          84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
          98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
         112, 113, 114, 115, 116, 117, 118, 119, 120, 121],
         ], device='cuda:0')
prev_a[torch.arange(batch_size), veh]

D:\develop_tools\Anaconda3\envs\fleet_rs\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([16, 20], device='cuda:0')

In [13]:
mask = torch.tensor([1, 1, 1])
min((1 - mask).nonzero().shape) == 0

True

In [25]:
visited = torch.tensor([[[1, 1, 0, 0, 0, 0]]],
       device='cuda:0', dtype=torch.uint8)
to_visit = torch.tensor([[[1, 0, 0, 1, 1, 1],
          [0, 1, 0, 1, 1, 1],
          [0, 0, 1, 1, 1, 1]]],
       device='cuda:0', dtype=torch.uint8)
visited.size(), to_visit.size()
if visited[0, :, (1 - to_visit[0, 1, :]).nonzero()].all():
    print(True)
else:
    print(False)
(1 - to_visit[0, 0, :]).nonzero().squeeze(0)

False


tensor([[1],
        [2]], device='cuda:0')

In [35]:
visited_loc = torch.tensor([[[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
          0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]]],
       device='cuda:0', dtype=torch.uint8)
to_visit = torch.tensor([[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], # 23个
          [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]],
       device='cuda:0', dtype=torch.uint8)
visited_loc.size(), to_visit.size()
for j in range(2):
    print((1 - to_visit[0, j, :]).nonzero().squeeze(0))
    print(visited_loc[0, :, (1 - to_visit[0, j, :]).nonzero().squeeze(0)])
    if visited_loc[0, :, (1 - to_visit[0, j, :]).nonzero().squeeze(0)].all():
        print("mask 了")
    else:
        print('没mask')

tensor([[19],
        [39],
        [59]], device='cuda:0')
tensor([[[1],
         [1],
         [1]]], device='cuda:0', dtype=torch.uint8)
mask 了
tensor([[19],
        [39],
        [59]], device='cuda:0')
tensor([[[1],
         [1],
         [1]]], device='cuda:0', dtype=torch.uint8)
mask 了


In [3]:
id = [459686, 12645, 201126, 324284, 261858, 41095, 251989, 206635, 189606, 56415, 384120, 235595, 72323, 269839, 469617, 85706, 246488, 466194, 210468, 258975, 199980, 239452, 301387, 332843, 211398, 74754, 330661, 392623, 214411, 310170, 313828, 64317, 31063, 162402, 274960, 279415, 380486, 444456, 229058, 373007, 415369, 168589, 315096, 362154, 93506, 145059, 411935, 339385, 22946, 91509, 79199, 303888, 1947, 176629, 117966, 278466, 73917, 312484, 281945, 34987, 45636, 372317, 437254, 87620, 63247, 373006, 224596, 422238, 391628, 57379, 123043, 234566, 50665, 46688, 465306, 397618, 404850, 470529, 149330, 1946, 416352, 283885, 460740, 88472, 282900, 54076, 163445, 205671, 475043, 37198, 440428, 170807, 257892, 164196, 113479, 368142, 317172, 352360]
len(id)
id.index(29687)

ValueError: 29687 is not in list